# Part 1 - Exploratory Data Analysis (EDA)

We will start with an Exploratory Data Analysis (EDA) of our SF housing dataset.  
It is always a good idea to start with an EDA before designing and training a machine learning algorithm.  
EDA gives us better insight to the data by using statistical and visualization techniques.  

Upon completing this notebook, we should have:  
* Familiarity with [Pandas] and [NumPy] for data management and analysis
* Familiarity with [Matplotlib] and [seaborn] for visualization
* A decent understanding of the characteristics of our dataset
[Pandas]: https://pandas.pydata.org/
[NumPy]: http://www.numpy.org/
[Matplotlib]: https://matplotlib.org/
[seaborn]: https://seaborn.pydata.org/

In [ ]:
import glob
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from geopy import Nominatim
import geojson
import folium
from branca.colormap import LinearColormap, StepColormap

%matplotlib inline

## Let's start by loading the data and have a peek at the contents 
The data was scraped from [zillow.com](https://www.zillow.com/) and is dispersed between several csv files.  
We will use Pandas to load the csv files and concatenate them into a single DataFrame

In [ ]:
all_csvs = []
# load the csv files from all scraping runs
for filename in glob.glob('./data/sf/**/*.csv'):
    all_csvs.append(pd.read_csv(filename))
# combine all dataframes together and drop any duplicate entries
df_raw = pd.concat(all_csvs, ignore_index=True).drop_duplicates()
print(f"Found a total of {len(df_raw)} data points")
# save this combined dataframe as csv for safe keeping
df_raw.to_csv('./data/sf/data_raw.csv', index=False)
# display first 5 entries of DataFrame
df_raw.head(5) 

### Our data is now contained in a variable named `df_raw` which is a pandas DataFrame.

## Reminder: Let's start the `generate_latlng()` now so we don't have to wait 

## Display some quick stats about the DataFrame
DataFrame has a few built in functions we can call to get a quick summary of the data:  
* `info()` displays a count of all non-null objects and their datatypes  
* `describe()` calculates basic statistics about all numerical values in the DataFrame

In [ ]:
df_raw.info()
df_raw.describe()

Our `describe()` method does not yet have any numerical data to describe for us.

Notice that a lot of the columns which provide numerical data are not in a format ready for consumption.  
For instance the `price` columns contain `$` and `,` characters and the `facts and features` column has valuable information about number of beds, bath and square footage embedded within the text.  
  
Let's parse and format these columns.

In [ ]:
# copy our original dataframe for safe keeping. We will manipulate `df` instead
df = df_raw.copy()

## Reformat price column
Time to use some convenient Pandas functions such as `.apply()` to apply a user defined formatting function to all values in a column.  

Remove `$` and `,` characters and format as `int`.  
Also, some prices are represented as `$1K` and `$1M` so let's replace with `1000` and `1000000`

In [ ]:
import re
def format_price(price):
    """Remove all non-numerical"""
    price = str(price)
    multiply_factor = 1
    if 'M' in price:
        multiply_factor = 1e6
    elif 'K' in price:
        multiply_factor = 1e3
    non_decimal = re.compile(r'[^0-9\.]')
    price_num = None
    try:
        price_num = float(non_decimal.sub('', price))*multiply_factor
    except Exception as e:
#         print(f'error converting \"{price}\": {e}')
        pass
    finally:
        return price_num

# replace the values in the price column with the formatted price
df['price'] = df.price.apply(format_price)

## Parse `facts and features` column into multiple columns 
An example entry in this column: `3 bds , 2 ba , 1,520 sqft`  
Parse the text using comma followed by a space '`, `' as the delimiter so that we can still capture the comma in the square footage.

In [ ]:
# TODO: consider studio as 0 beds?
non_decimal = re.compile(r'[^\d.]+')
def parse_beds(string):
    strings = string.lower().split(', ')
    num_beds = None
    for s in strings:
        if "bd" in s:
            try:
                num_beds = float(non_decimal.sub('', s))
            except Exception as e:
                pass
        # treat studio as 0 bedrooms
        elif "studio" in s.lower():
            num_beds = 0
        return num_beds

def parse_bath(string):
    strings = string.lower().split(', ')
    num_bath = None
    for s in strings:
        if "ba" in s:
            try:
                num_bath = float(non_decimal.sub('', s))
            except Exception as e:
                pass
            finally:
                return num_bath
def parse_sqft(string):
    strings = string.lower().split(', ')
    sqft = None
    for s in strings:
        if "ft" in s:
            try:
                sqft = float(non_decimal.sub('', s))
            except Exception as e:
                pass
            finally:
                return sqft
df['bed'] = df['facts and features'].apply(parse_beds)
df['bath'] = df['facts and features'].apply(parse_bath)
df['sqft'] = df['facts and features'].apply(parse_sqft)

## Parse `title` column into `property_type`
The title of the posting contains some information we can parse. For instance we can map `'Condo For Sale'` --> `condo`

First let's see if there is a pattern to the titles:

In [ ]:
print(df.title.unique())
print(df.title.value_counts())

Looks like there is a limited amount of unique values, which is good. We can design our parser to catch most cases.  
We won't parse 'For Sale by Owner' since it is too vague

In [ ]:
# map property types
property_types = {'Condo For Sale': 'condo', 
                  'House For Sale': 'house', 
                  'Apartment For Sale': 'apartment', 
                  'New Construction': 'new',
                  'Foreclosure': 'foreclosure', 
                   'Lot/Land For Sale': 'lot', 
                  'Coming Soon': 'coming', 
                  'Co-op For Sale': 'coop',
                  'Auction': 'auction', 
                  'For Sale by Owner': None, 
                  'Townhouse For Sale': 'townhouse'}
def parse_property_type(string):
    try:
        property_type = property_types[string]
    except KeyError as e:
        print(e)
        property_type = None
    finally:
        return property_type
df['property_type'] = df['title'].apply(parse_property_type)

In [ ]:
df.property_type.value_counts()

If we check the `info()` of the dataframe, we should see some columns are now numerical (`float64`)

In [ ]:
df.info()

Now we can call `describe()` to get some stats about the numerical values

In [ ]:
df.describe()

### Very large maximum price albeit not suprising. 

In [ ]:
# describe only the 'price' column
df['price'].describe()

## We have the gist of the dataset size and its contents, it's time to go more in depth and Visualize the data.  
We will use `Seaborn` to visualize the data.

### Plot histogram of prices

In [ ]:
# globally set our seaborn plot size to 12 by 8 inches:
sns.set(rc={'figure.figsize':(12, 8)})

def plot_prices(df: pd.DataFrame, bins: list):
    fig, ax = plt.subplots()
    ax.set_xticks(bins)
    plt.xticks(rotation='vertical')
    return sns.distplot(df.price, bins=bins)

bins = range(int(df.price.min()), int(df.price.max()), 500000)
plot_prices(df.dropna(), bins)

### Definitely a skewed distribution, looks as if we have a few outliers at the higher range of the prices.  
### We can quantify how "non-normal" our distribution is by calculating:  
* `Skewness` - A measure of the symmetry (or lack thereof) of a distribution
* `Kurtosis` - Whether distrubition is "heavy-tailed" or "light-tailed" or in other words: how "sharp" the peak is.

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df['price'].skew())
print("Kurtosis: %f" % df['price'].kurt())

## Plot with outliers removed

In [ ]:
df_no_outliers = df[df.price < 8e6]
bins = range(int(df_no_outliers.price.min()),int(df_no_outliers.price.max()),500000)
plot_prices(df_no_outliers, bins)
print("Skewness (outliers removed): %f" % df_no_outliers['price'].skew())
print("Kurtosis (outliers removed): %f" % df_no_outliers['price'].kurt())

### Removing the outliers improved our skewness and kurtosis values.
We will remember this when cleaning the data for our model. Machine learning models work best with normally distributed data. Outliers may affect model performance.

## Plot missing values.
Recall that there were some columns which are incomplete. Plot a bar graph describing this:

In [ ]:
missing = df.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

Variables that are missing values can either be removed from the dataset or have their missing values replaced (perhaps with 0 or the mean of the column). Remember this for data cleaning.

## Visualize the house prices w.r.t. location with a slippy map  
We have some location information in the `address` column. We'll use geocoding to convert the string address to Lat Long.  

In [ ]:
import time
import random
from IPython.display import clear_output
from geopy.geocoders import Nominatim
NUM_RETRIES = 3 # number of retries for request
def generate_latlng(dataframe: pd.DataFrame):
    dataframe = dataframe.copy()
    geocoder = Nominatim()
    latlngs = []
    for address, city in zip(dataframe.address, dataframe.city):
        time.sleep(int(random.randint(0,6)/3)) # try not to get your ip blacklisted by Nominatim
        clear_output(wait=True)
        location = None
        for i in range(NUM_RETRIES):
            try:
                location = geocoder.geocode(f'{address} {city}')
                break
            except Exception as e:
                print(f"Error: {e}")
        if location:
            latlngs.append((location.latitude, location.longitude))
        else:
            latlngs.append(None)
        print(f'{len(latlngs)+1}/{len(dataframe)} complete...')
    dataframe['latlng'] = latlngs
    return dataframe

The above function `generate_latlng()` takes a while to run since we call a web service `Nominatim` to perform the address --> latlng conversion.  
Let's use Python's `multiprocessing` module to run these conversions in a background job.

In [ ]:
# alternatively uncomment here to load pre-processed data
# df = pd.read_csv('./data/sf/data_w_latlng.csv')

In [ ]:
from multiprocessing import Pool
pool = Pool()
job = pool.apply_async(func=generate_latlng, args=(df,))

We can check whether the job is complete using `ready()`  

In [ ]:
if job.ready():
    df = job.get()
    df.to_csv('./data/sf/data_w_latlng.csv')
else:
    print("job not complete yet")

In [ ]:
df.head(10)

We use `folium` to render the slippy map in the notebook.  
Note that there are hundreds of houses to be displayed and this requires a fair bit of RAM. If your browser crashes you can adjust the amount to be displayed by changing the variable `display_max`.

In [ ]:
def draw_houses_on_map(dataframe: pd.DataFrame):
    dataframe = dataframe.copy()
    # create a folium map object centered in SF
    m = folium.Map(location=(37.7, -122.4))
    # create a colormap of the prices (we limit prices between 5e5 and 10e6)
    colors = ['gray', 'green','blue','red','orange', 'yellow']
    min_price, max_price = 5e5, 6e6
    colormap = StepColormap(colors=colors,vmin=min_price, vmax=max_price, caption='price')
    m.add_child(colormap)
    # amount of points to render on the map. WARNING: significant RAM required to plot all points and may crash your browser 
    display_max = len(dataframe) # plot all
    # display_max = 100 # uncomment and adjust this number if needed
    displayed = 0
    for i, latlng in zip(dataframe.index, dataframe['latlng']):
        price = dataframe.loc[i, 'price']
        if latlng is not None:
            if isinstance(latlng, str):
                lat, lng = latlng.replace('(','').replace(')','').split(',')
                latlng = (float(lat), float(lng))
            if not isinstance(latlng, tuple):
                continue
            style = {'fillColor': colormap(price),
                    'color' : colormap(price)}
            p = geojson.Point(coordinates=(latlng[1], latlng[0]), style=style)
            # build an HTML string to be displayed if we click a marker.
            html_info = '<li>Price: ${}</li><li>Property Type: {}</li>'.format(dataframe.loc[i, 'price'], dataframe.loc[i, 'property_type'])
            m.add_child(folium.Marker(location=latlng, icon=folium.Icon(color='black', icon_color=colormap(price)), popup=folium.Popup(html=html_info)))
            displayed += 1
            if displayed > display_max:
                break
    return m
draw_houses_on_map(df)

We can observe some patterns w.r.t. location.  
Seems the more expensive homes are Central and North and the "lower" (finger quotes) priced homes on the outside

## Next, let's see how some of the variables interact with the list price.  
Since `price` is our target variable (the variable we are trying to predict), it is useful visualize how each variable relates to `price`. 

### sqft
Total square footage

In [ ]:
# sqft/saleprice
var = 'sqft'
sns.regplot(df[var], df['price'], )

The relationship looks linear with some spreading as sqft increases. We can also see there are some houses with almost zero square feet! Let's investigate why:  
  
Note on `pandas.DataFrame` indexing:  
* `df['sqft'] < condition` gives us a "truth array" where True values match the condition and False otherwise. If we index the original DataFrame with this truth array we get a filtered result

In [ ]:
# filter the DataFrame with nearly zero sqft
df[df['sqft'] < 10].head()

Looks like we have some bad data from the web scraping. We will remember to remove these when we get to our data cleaning notebook

### bed
Number of bedrooms

In [ ]:
var = 'bed'
sns.regplot(df[var], df['price'], )

We observe a bit of a positive correlation between price and number of beds

### bath
Number of bathrooms

In [ ]:
var = 'bath'
sns.regplot(df[var], df['price'], )

Positive correlation between number of baths and price

## Generate a correlation matrix

A correlation matrix will graphically show us which variables are most correlated to our target variable `price`.  
A positive correlation (w.r.t. price) means as the variable increases, the price increases

In [ ]:
corrmat = df.corr()
sns.heatmap(corrmat, vmax=1, square=True);

Observe the `price` across a row or column to get an idea which variables are most likely to correlate to price.

## Categorical Variables.  
So far we have only dealt with numeric variables however there are several non-numerical (**Categorical**) variables to be investigated as well:  
  
Categorical variables are ones which provide information but are not quantified numerically. For instance, the `postal_code` variable gives us information about which neighbourhood the house is located. We found from our map plot that this data may be useful for predicting `price`.  
  
In order to use these categorical variables in our model, we encode them into a numerical representation called a [Dummy Variable]. We cover Dummy Variables in a later notebook.
[Dummy Variable]: https://en.wikipedia.org/wiki/Dummy_variable_(statistics)

In [ ]:
print(df.columns)

Let's choose `property_type`, and `postal_code` to investigate.  
We can use the `unique()` function on the categorical columns to see the different categories.

In [ ]:
print(df['postal_code'].value_counts())
print(df['property_type'].value_counts())

There are some variables with only a single value, let's get rid of that data.

In [ ]:
postal_codes = [
'94530',
'94014',
'94608',
'94607',
'94005',
'94706',
'94501'
]
for postal_code in postal_codes:
    df = df[df.postal_code != int(postal_code)]
property_types = ['townhouse', 'foreclosure']
for property_type in property_types:
    df = df[df.property_type != property_type]
print(df['postal_code'].value_counts())
print(df['property_type'].value_counts())

Visualize these categories as box plots.  
We use the `pandas.melt()` function to flatten our variables into a single column so we can plot.  
The result of using `melt()` is most easily understood by displaying the result.

In [ ]:
vars_to_analyze = ['property_type', 'postal_code']
df_melt = pd.melt(df, id_vars=['price'], value_vars=vars_to_analyze)
for var in vars_to_analyze:
    df_var = df_melt[df_melt['variable'] == var]
    sns.boxplot(x=df_var['value'], y=df_var['price'])
    x=plt.xticks(rotation=45)
    plt.title(var)
    plt.show()

In [ ]:
# uncomment to see the effects of melt()
# df_melt.head(20)
# df.head(20)

## Analysis of variance (ANOVA)
We use ANOVA to explore how much variance occurs **between** groups (ie. *[price vs property_type]* vs *[price vs postal_code]*) versus how much variance occurs **within** each group (ie *[price vs sub_area]* alone).  
In the end this tells us is how useful it will be to group `price` into these 4 groups (and if including each variable in our model is useful to us).  
Here's a quick YouTube video that may better explain ANOVA:  

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo(id='ITf4vHhyGpc')

In [ ]:
def anova(df):
    anv = pd.DataFrame()
    anv['feature'] = vars_to_analyze
    pvals = []
#     import pdb; pdb.set_trace()
    for c in vars_to_analyze:
        samples = []
        for cls in df[c].unique():
            s = df[df[c] == cls]['price'].values
            samples.append(s)
        try:
            pval = stats.f_oneway(*samples)[1]
        except Exception as e:
            pval=None
            print(e)
        finally:
            pvals.append(pval)
    anv['pval'] = pvals
    return anv.sort_values('pval')

a = anova(df.dropna())
a['disparity'] = np.log(1./a['pval'].values)
sns.barplot(data=a, x='feature', y='disparity')
x=plt.xticks(rotation=45)

This gives us a rough estimate of effect each variable will have on our model.

# Save our DataFrame to .csv

In [ ]:
df.to_csv('./data/sf/data.csv', index=False)

## Hopefully the EDA has improved our intuition about the dataset. Now we can move onto data cleaning!